# Install Event Hub packages

In [ ]:
! python --version

! pip install azure-eventhub==5.11.5 azure-identity --upgrade --force --quiet

StatementMeta(, , -1, Waiting, , Waiting)

# Connect to Event Stream

## Azure Managed Identity Setup Requirements

**For this to work, you need to:**

1. **Enable Managed Identity** on your Azure service:
   - Azure App Service: Go to Identity → System assigned → On
   - Azure Function App: Go to Identity → System assigned → On  
   - Azure Kubernetes Service (AKS): Use Azure AD Pod Identity or Workload Identity
   - Azure VM: Enable system-assigned managed identity

2. **Grant permissions** to your Event Hub:
   - Go to your Event Hub Namespace → Access control (IAM)
   - Add role assignment: "Azure Event Hubs Data Sender"
   - Assign to your managed identity

3. **For local development:**
   - Install Azure CLI: `az login` 
   - Or use Visual Studio/VS Code Azure authentication
   - `DefaultAzureCredential()` will automatically use your local credentials

**No connection strings or secrets needed!** 🔒

In [ ]:
import json
import os
import socket
import random
from random import randrange
from azure.eventhub import EventHubProducerClient, EventData
from azure.identity import DefaultAzureCredential

# Azure Managed Identity Configuration (Secure - No hardcoded secrets)
# Note: This health notebook uses a different Event Hub than the other notebooks
eventhub_namespace = "esehmtcyxemzzhm8ha9l9aso.servicebus.windows.net"
eventhub_name = "es_553da34c-8ede-46e5-9983-403ca1417cd1"

# Use Azure Managed Identity for authentication
credential = DefaultAzureCredential()
producer_events = EventHubProducerClient(
    fully_qualified_namespace=eventhub_namespace,
    eventhub_name=eventhub_name,
    credential=credential
)

hostname = socket.gethostname()
print(f"Connected to Health Event Hub using Azure Managed Identity from {hostname}")
print(f"Event Hub: {eventhub_name}")
print(f"Namespace: {eventhub_namespace}")

StatementMeta(, , -1, Waiting, , Waiting)

In [ ]:
# Test the Managed Identity connection for Health Event Hub
try:
    # Try to create a test batch to verify connection
    test_batch = producer_events.create_batch()
    print("✅ Azure Managed Identity authentication successful!")
    print(f"✅ Connected to Health Event Hub: {eventhub_name}")
    print(f"✅ Namespace: {eventhub_namespace}")
    print("✅ Ready to send health/hospital data events securely")
except Exception as e:
    print("❌ Managed Identity authentication failed!")
    print(f"Error: {e}")
    print("\n🔧 Troubleshooting steps:")
    print("1. Ensure Managed Identity is enabled on your Azure service")
    print("2. Grant 'Azure Event Hubs Data Sender' role to the managed identity")
    print("3. For local development, run 'az login' first")
    print("4. Check that the Event Hub namespace and name are correct")
    print("5. Verify this health Event Hub exists and is accessible")

# Send events to Event Stream

In [ ]:
def sendToEventsHub(jsonEvent, producer):
    eventString = json.dumps(jsonEvent)
    #print(eventString) 
    event_data_batch = producer.create_batch() 
    event_data_batch.add(EventData(eventString)) 
    producer.send_batch(event_data_batch)

StatementMeta(, , -1, Waiting, , Waiting)

# Wrapper to generate events and send to event Hub

In [ ]:
Hospital_Departments = {
  "Cardiology": {
    "department_code": "DGBW"
  },
  "Neurology": {
    "department_code": "H0RU"
  },
  "Oncology": {
    "department_code": "76VC"
  },
  "Pediatrics": {
    "department_code": "A9KZ"
  },
  "Radiology": {
    "department_code": "CR8E"
  },
  "Orthopedics": {
    "department_code": "VUG0"
  },
  "Emergency": {
    "department_code": "663P"
  },
  "Dermatology": {
    "department_code": "8450"
  },
  "Gastroenterology": {
    "department_code": "Z8IG"
  },
  "Urology": {
    "department_code": "72RR",
  }
}

StatementMeta(, , -1, Waiting, , Waiting)

In [ ]:
import random, math, json
import random
import time
import uuid
from datetime import datetime, timedelta

class ContosoHospitalSimulator:
    def __init__(self, current_time_pas):
        ## self.grid_id = grid_id
        self.unique_id = str(uuid.uuid4())
        self.current_time = current_time_pas

    def log_event(self, activity, sensor_data):
        event = {
            "timestamp": self.current_time.strftime("%Y-%m-%d %H:%M:%S"),
            "event_id": self.unique_id,
            "source": activity,
            # "grid_id": grid_id,
            "sensor_data": sensor_data
        }
        log_event = event
        ##log_obj = json.dumps(log_event)
        #impressionEvent = json_object    
        sendToEventsHub(log_event, producer_events)
        #print(log_event)

    def get_sensor_data(self, source, Department, details):
        #sources = ["United Care-HMO", "Medicare", "Athens-HMO ", "Self Pay", "Workers Comp"]
        data = {}

        # for source in sources:
        Billed_Amount = round(random.uniform(300, 500) if random.random() < 1/1000 else random.uniform(100, 300), 0)

        # Ensure Collection Amount < Billed_Amount only 1 in 10 times
        if random.random() > 1/20:
            Collection_Amount = round(random.uniform(0.5 * Billed_Amount, Billed_Amount), 0)
        else:
            Collection_Amount = round(random.uniform(1.1 * Billed_Amount, 2 * Billed_Amount), 0)

        # Calculate price based on the difference
        percentage = round((Collection_Amount / Billed_Amount) * 100, 0)

        if percentage < 70:
            price = round(random.uniform(-10, 30), 0)
        elif percentage < 80:
            price = round(random.uniform(30, 50), 0)
        elif percentage < 90:
            price = round(random.uniform(50, 60), 0)
        elif percentage < 100:
            price = round(random.uniform(60, 100), 0)
        else:
            # print("greater")
            price = round(random.uniform((100 + ((percentage-100)/5)*10), 300), 0)

        if source in ["Self Pay", "Workers Comp"]:
            base_claims = 20
            claims_multiplier = math.floor(max((Billed_Amount - 50),0) / 10)
            Claims_Rejections = round(base_claims + claims_multiplier * 10, 0)    
        else:
            base_claims = 50
            claims_multiplier = math.floor(max((Billed_Amount - 50),0) / 10)
            Claims_Rejections = round(base_claims + claims_multiplier * 5, 0)

        #print(Billed_Amount, Collection_Amount, percentage, price, Claims_Rejections)

        data = {
            "Department": Department,
            "department_code":details['department_code'],
            "base_claims":base_claims,
            "Billed_Amount": Billed_Amount,
            "Collection_Amount": Collection_Amount,
            "Claims_Rejections": Claims_Rejections,
        }

        return data

def simulate_hospital_data():
    start_time = datetime.now() - timedelta(days=3)
    counter = 0

    while True:
        for Department, details in Hospital_Departments.items():
            current_time_pas = start_time + timedelta(seconds=80 * counter)
            grid = ContosoHospitalSimulator(current_time_pas)

            for src in ["United Care-HMO", "Medicare", "Athens-HMO ", "Self Pay", "Workers Comp"]:
                sensor_data = grid.get_sensor_data(src, Department, details)
                grid.log_event(src, sensor_data)

            counter += 1
            print(f"Counter: {counter}, Time: {current_time_pas}, Department: {Department}")

            # Determine if we need to sleep
            if current_time_pas >= datetime.now():
                time.sleep(80)
            else:
                time.sleep(1)
simulate_hospital_data()

StatementMeta(, , -1, Waiting, , Waiting)